# Report for Navigation Project

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment
Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.7 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# please do not modify the line below
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Description of the  Learning Algorithm

So the Learnning Algo. is based on :
* Q learning with Epislon greedy policy with decay
* An underlying NN (defined later below)
* Imporvments used to enhance the Deep Q-Networks (DQN) such as :
    * Replay Buffer
    * Fixed Targets

#### 2.1 First lets Explore the Agent

In [5]:
from dqn_agent import Agent
from collections import deque
import matplotlib.pyplot as plt
import torch
%matplotlib inline
action_size = brain.vector_action_space_size
state_size = brain.vector_observation_space_size
agent = Agent(state_size=state_size, action_size=action_size, seed=0)

In [10]:
#Load Weights To Model
agent.qnetwork_local.load_state_dict(torch.load("qnetwork_local.pth"))
agent.qnetwork_local.eval()

agent.qnetwork_target.load_state_dict(torch.load("qnetwork_target.pth"))
agent.qnetwork_target.eval()

QNetwork(
  (fc1): Linear(in_features=37, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)

Bellow is the the Deep Q Network which is compromised of 2 Networks (MainQ, target)


In [9]:
print("\n=============== This is the Main Q Network ===============")
print(agent.qnetwork_local)\

print("\n=============== This is the Main TARGET Network ===============")
print(agent.qnetwork_target)

print("\n=============== This is the STORAGE used for Replay Buffer of max size  ===============")
print (agent.memory.memory.maxlen)


=============== This is the Main Q Network ===============
QNetwork(
  (fc1): Linear(in_features=37, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)

=============== This is the Main TARGET Network ===============
QNetwork(
  (fc1): Linear(in_features=37, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)

=============== This is the STORAGE used for Replay Buffer of max size  ===============
100000


Below is Q-Learning Hyper-Parameters used in DQN :
* BUFFER_SIZE = int(1e5)  # replay buffer size
* BATCH_SIZE = 64         # minibatch size
* GAMMA = 0.99            # discount factor
* TAU = 1e-3              # for soft update of target parameters
* LR = 5e-4               # learning rate 
* UPDATE_EVERY = 4        # how often to update the network

Adam was used as optimizer for gradient descent.


### 3. Testing Agent Against Success Criteria

In [7]:
def TestAgent(n_episodes=2000, max_t=1000, MinScore=13.000001, MinEposides=100):
    scores = []          # list containing scores from each episode
    SuccCritCounter = 0  # Criteria of success, at least 100 consecutive eposides of at least score of 13
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        score = 0
        for t in range(max_t):
            action = agent.act(state) #get action to perform
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            state = next_state 
            score += reward    
            if done:
                break 
        scores.append(score)              # save most recent score
        print('\rEpisode {}\Score: {}\Consecutive Success: {}'.format(i_episode, score,SuccCritCounter), end="")

        if(score<MinScore) :
            SuccCritCounter=0 # Reset Counter
        else: 
            SuccCritCounter+=1
            if(SuccCritCounter>=MinEposides): #Criteria satisfied 
                break
    return scores

In [10]:
scores = TestAgent()# get Testing scores

Episode 2000\Score: 13.0\Consecutive Success: 82

####  3.1 Plot of Rewards
Below is graph of Rewards and a line at score 13 to show relatively the perfomance of the agent against the 1st success criteria, ie the minScore

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores) #scores
plt.plot([0,len(scores)], [13,13]) # Criteria Min Value
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

####  3.2 Plot of Consecutive Success 
Below is graph of Rewards and a line at score 13 to show relatively the perfomance of the agent against the 1st success criteria, ie the minScore

In [33]:
def NumberOfConseqSuccess(scores,MinScore=13.000001):
    line=[]
    counter=0
    for s in scores:
        if(s<MinScore):
            counter=0
        else:
            counter+=1
        line.append(counter)
    return line

In [45]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot([0,len(scores)], [100,100]) # Criteria Min #eposides
plt.plot(np.arange(len(scores)), NumberOfConseqSuccess(scores,13.00001)) #Criteria Min Eposides
plt.ylabel('#Consecutive, score>=13')
plt.xlabel('Episode #')
plt.show()

### Ideas for Future Work

When finished, you can close the environment.

In [ ]:
env.close()